In [1]:
import pandas as pd
import numpy as np
import collections
import re
import sqlite3

In [2]:
# Change this to your path to the DB file!
conn = sqlite3.connect("/path/to/flights.db")

### Load raw data 

In [3]:
# load our datasets
# These first four are used as lookup tables
airlines =     pd.read_sql('SELECT * FROM airlines', con=conn)
airports =     pd.read_sql('SELECT * FROM airports', con=conn)
cancel_codes = pd.read_sql('SELECT * FROM cancel_codes', con=conn)
carriers =     pd.read_sql('SELECT * FROM carriers', con=conn)
# This has all the data about flights
flights =      pd.read_sql('SELECT * FROM flights', con=conn)

In [ ]:
#Closing the connection to the db, since we're done using it
conn.close()

### Some transforms to get it ready for analysis

In [4]:
# Turn our flights table into a list of dicts
# Normally we wouldn't do this, but for the sake of learning to deal with data
# structures in Python, we'll do it this way :)
flights = flights.to_dict(orient='records')

### HW 1: Answer the following questions about the `flights` :

What datatype is our `flights` object now (a list, dict, etc)?

How many elements are in it?

How can we look at/inspect the first element?

### HW 2: Create dictionaries from our lookup tables, making the `Code` the key and the `Description` the value

In [8]:
cancel_codes_dict = dict(zip(cancel_codes.Code, cancel_codes.Description))

# Go ahead and create more dictionaries for airlines, airports and carriers where
# for each the key will be the `Code` and the value will be the `Description`

# When you're done there should be four dicts (cancel_codes_dict, airlines_dict, airports_dict, carriers_dict)

### HW 3: Add in our dictionary mappings to our `flights` data
Using our mappings, please create the following new keys in each 'row' of flights with the appropriate value:
 - AIRLINE_NAME
 - ORIGIN_AIRPORT_NAME
 - CARRIER_NAME
 - CANCELLATION_REASON

In [10]:
for row_dict in flights:
    # Do stuff here!
    ...